In [7]:
import pandas as pd
import psycopg2
import datetime
import os
from sqlalchemy import create_engine
import logging
import pickle
import time
import requests
from dateutil import parser
pd.set_option('display.max_columns', None)

db_host = 'localhost'
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')

engine = create_engine("postgresql+psycopg2://{user}:{pw}@{host}/{db}".format(host=db_host, db=db_name, user=db_user, pw=db_password))
day_candles = pd.read_sql_query(f"select * from candle60 where date(datetime) = '2024-07-09' order by datetime", con=engine)

previous_day_candles = pd.read_sql_query(f"select * from candle60 where date(datetime) = '2024-07-08' order by datetime", con=engine)
idx = previous_day_candles.groupby('symbol')['datetime'].idxmax()
max_datetime_per_symbol = previous_day_candles.loc[idx]
previous_candles = max_datetime_per_symbol[["token", "symbol", "close"]].reset_index(drop=True)

distinct_datetimes = day_candles['datetime'].unique()

In [8]:
previous_candles

,token,symbol,close
0,13061,360ONE,983.05
1,474,3MINDIA,39276.00
2,4481,AARTIDRUGS,554.00
3,7,AARTIIND,710.00
4,5385,AAVAS,1796.80
...,...,...,...
489,1076,ZENSARTECH,735.00
490,16915,ZFCVINDIA,15750.00
491,5097,ZOMATO,207.50
492,7929,ZYDUSLIFE,1159.50


In [9]:
current_candles = day_candles[day_candles['datetime'].dt.time == datetime.time(9, 15, 0)]
current_candles

,token,symbol,datetime,open,high,low,close,volume
0,7,AARTIIND,2024-07-09 09:15:00,708.35,708.95,706.20,707.30,9837
1,13,ABB,2024-07-09 09:15:00,8614.90,8619.55,8572.65,8580.00,4035
2,22,ACC,2024-07-09 09:15:00,2673.05,2686.75,2662.00,2680.10,1664
3,25,ADANIENT,2024-07-09 09:15:00,3115.95,3128.85,3101.70,3122.95,19142
4,40,AEGISCHEM,2024-07-09 09:15:00,927.50,946.25,927.50,944.00,11050
...,...,...,...,...,...,...,...,...
489,30125,IBULHSGFIN,2024-07-09 09:15:00,168.89,169.50,168.61,168.61,114731
490,31163,RHIM,2024-07-09 09:15:00,616.05,616.10,614.00,614.05,1852
491,31181,MCX,2024-07-09 09:15:00,3939.65,3952.75,3903.10,3943.90,14754
492,31415,NBCC,2024-07-09 09:15:00,192.20,192.86,191.78,191.99,1042152


In [10]:
def find_initial_positive_trend(row):
    initial_cutoff_percent = 0.4
    if row['change'] > row['open']*initial_cutoff_percent/100:
        return [True, row['datetime'], None, row['close_previous'], row['close_current'], row['change_percentage'], row['volume'], 1, 0]
    else:
        return [False, None, None, None, None, None, None, 0, 1]

trend_summary = None
if current_candles['datetime'].dt.time.iloc[0] == pd.to_datetime('09:15:00').time():
    trend_summary = pd.merge(current_candles, previous_candles, how = 'inner', on=['token','symbol'], suffixes=('_current', '_previous'))
    trend_summary['change'] = trend_summary['close_current'] - trend_summary['close_previous']
    trend_summary['change_percentage'] = round((trend_summary['change']/trend_summary['close_previous'])*100,2)
    trend_summary['half'] = (trend_summary['open'] + trend_summary['close_current'])/2
    trend_summary['pause'] = False
    trend_summary['trend_props'] = trend_summary.apply(find_initial_positive_trend, axis=1)
    trend_summary[['trend', 'start_datetime', 'end_datetime', 'start_price', 'end_price', 'trend_change', 'trend_volume', 'trend_candle_count', 'stag_count']] = pd.DataFrame(trend_summary['trend_props'].tolist(), index=trend_summary.index)
    trend_summary = trend_summary.drop(columns=['open', 'high', 'low', 'volume', 'trend_props', 'change'])
    trend_summary.rename(columns={'close_current': 'previous_close', 'close_previous':'day_open', 'change_percentage':'day_change'}, inplace=True)

trend_summary

,token,symbol,datetime,previous_close,day_open,day_change,half,pause,trend,start_datetime,end_datetime,start_price,end_price,trend_change,trend_volume,trend_candle_count,stag_count
0,7,AARTIIND,2024-07-09 09:15:00,707.30,710.00,-0.38,707.825,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1
1,13,ABB,2024-07-09 09:15:00,8580.00,8568.40,0.14,8597.450,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1
2,22,ACC,2024-07-09 09:15:00,2680.10,2680.00,0.00,2676.575,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1
3,25,ADANIENT,2024-07-09 09:15:00,3122.95,3110.00,0.42,3119.450,False,True,2024-07-09 09:15:00,None,3110.00,3122.95,0.42,19142.0,1,0
4,40,AEGISCHEM,2024-07-09 09:15:00,944.00,930.80,1.42,935.750,False,True,2024-07-09 09:15:00,None,930.80,944.00,1.42,11050.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,30125,IBULHSGFIN,2024-07-09 09:15:00,168.61,167.95,0.39,168.750,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1
490,31163,RHIM,2024-07-09 09:15:00,614.05,614.50,-0.07,615.050,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1
491,31181,MCX,2024-07-09 09:15:00,3943.90,3955.00,-0.28,3941.775,False,False,NaT,None,NaN,NaN,NaN,NaN,0,1
492,31415,NBCC,2024-07-09 09:15:00,191.99,190.75,0.65,192.095,False,True,2024-07-09 09:15:00,None,190.75,191.99,0.65,1042152.0,1,0


In [11]:
current_candles = day_candles[day_candles['datetime'].dt.time == datetime.time(9, 16, 0)]
current_candles

,token,symbol,datetime,open,high,low,close,volume
494,7,AARTIIND,2024-07-09 09:16:00,707.55,709.10,707.00,708.95,2494
495,13,ABB,2024-07-09 09:16:00,8571.80,8594.25,8570.20,8594.20,1719
496,22,ACC,2024-07-09 09:16:00,2680.10,2685.00,2677.75,2684.75,769
497,25,ADANIENT,2024-07-09 09:16:00,3122.25,3132.00,3122.00,3129.45,6532
498,40,AEGISCHEM,2024-07-09 09:16:00,943.55,951.80,943.55,951.80,13495
...,...,...,...,...,...,...,...,...
948,30108,ABFRL,2024-07-09 09:16:00,323.35,323.90,322.80,323.00,25754
949,30125,IBULHSGFIN,2024-07-09 09:16:00,168.99,169.69,168.99,169.51,66036
950,31181,MCX,2024-07-09 09:16:00,3940.00,3949.90,3935.55,3949.15,2506
951,31415,NBCC,2024-07-09 09:16:00,191.98,192.75,191.85,192.00,459804


In [12]:
def find_positive_trend(row):
    retracement_threshold = 0.7
    stagnation_percent = 0.3
    stag_threshold = 10
    current_change_percentage = round((row['close']-row['start_price']/row['start_price'])*100,2)
    trend_change = row['trend_change']
    new_volume = row['trend_volume'] + row['volume']
    half = row['half']
    new_half = (row['open'] + row['close'])/2

    if abs(new_half-half) > half*stagnation_percent/100:
        stag_count = 0
        row['half'] = new_half
    else:
        stag_count = row['stag_count'] + 1

    if row['trend'] == True:
        if current_change_percentage < retracement_threshold*trend_change or row['datetime_current'].time() == datetime.time(15, 29, 0) or stag_count > stag_threshold:
            return[False, False, None, None, None, None, None, None, 0, stag_count, row['half']]
        elif current_change_percentage > trend_change:
            return[False, True, row["start_datetime"], row["datetime_current"], row["start_price"], row["close"], current_change_percentage, new_volume, row["trend_candle_count"]+1, stag_count, row['half']]
        elif current_change_percentage > retracement_threshold*trend_change and stag_count <= stag_threshold:
            return[True, True, row["start_datetime"], row["end_datetime"], row["start_price"], row["end_price"], trend_change, row["trend_volume"], row["trend_candle_count"], stag_count, row['half']]
        
    else:
        if row['change_percentage'] > row['open']*stagnation_percent/100:
            return [False, True, row['datetime_current'], row['datetime_current'], row['open'], row['close'], row['change_percentage'], row['volume'], 1, stag_count, row['half']]
        else:
            return [False, False, None, None, None, None, None, None, 0, stag_count, row['half']]
    
trend_summary = pd.merge(trend_summary, current_candles, how = 'inner', on=['token','symbol'], suffixes=('_previous','_current'))
trend_summary['change_percentage'] = round((trend_summary['close'] - trend_summary['previous_close'])*100/trend_summary['previous_close'],2)
trend_summary['net_change'] = trend_summary['close'] - trend_summary['day_open']
trend_summary['day_change'] = round((trend_summary['net_change']/trend_summary['day_open'])*100,2)
trend_summary['half'] = (trend_summary['open'] + trend_summary['close'])/2
trend_summary['pause'] = False
trend_summary['trend_props'] = trend_summary.apply(find_positive_trend, axis=1)
trend_summary[['pause', 'trend', 'start_datetime', 'end_datetime', 'start_price', 'end_price', 'trend_change', 'trend_volume', 'trend_candle_count', 'stag_count', 'half']] = pd.DataFrame(trend_summary['trend_props'].tolist(), index=trend_summary.index)
trend_summary = trend_summary.drop(columns=['open', 'high', 'low', 'volume', 'trend_props', 'net_change','change_percentage','previous_close','datetime_previous'])
trend_summary.rename(columns={'close': 'previous_close', 'datetime_current': 'datetime'}, inplace=True)

trend_summary

,token,symbol,day_open,day_change,half,pause,trend,start_datetime,end_datetime,start_price,end_price,trend_change,trend_volume,trend_candle_count,stag_count,datetime,previous_close
0,7,AARTIIND,710.00,-0.15,708.250,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,2,2024-07-09 09:16:00,708.95
1,13,ABB,8568.40,0.30,8583.000,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,2,2024-07-09 09:16:00,8594.20
2,22,ACC,2680.00,0.18,2682.425,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,2,2024-07-09 09:16:00,2684.75
3,25,ADANIENT,3110.00,0.63,3125.850,False,True,2024-07-09 09:15:00,2024-07-09 09:16:00,3110.00,3129.45,312845.00,25674.0,2,1,2024-07-09 09:16:00,3129.45
4,40,AEGISCHEM,930.80,2.26,947.675,False,True,2024-07-09 09:15:00,2024-07-09 09:16:00,930.80,951.80,95080.00,24545.0,2,1,2024-07-09 09:16:00,951.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,30108,ABFRL,322.75,0.08,323.175,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,2,2024-07-09 09:16:00,323.00
455,30125,IBULHSGFIN,167.95,0.93,169.250,False,True,2024-07-09 09:16:00,2024-07-09 09:16:00,168.99,169.51,0.53,66036.0,1,2,2024-07-09 09:16:00,169.51
456,31181,MCX,3955.00,-0.15,3944.575,False,False,NaT,NaT,NaN,NaN,NaN,NaN,0,2,2024-07-09 09:16:00,3949.15
457,31415,NBCC,190.75,0.66,191.990,False,True,2024-07-09 09:15:00,2024-07-09 09:16:00,190.75,192.00,19100.00,1501956.0,2,1,2024-07-09 09:16:00,192.00
